In [ ]:
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule

class MDBSolosClassifier(LightningModule):
    def __init__(self):
        super().__init__()
  
#         self.jtfs_bn
#         self.jtfs
#         self.conv = efficientnet

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def forward(self, x):
        Sx = self.jtfs_bn(self.jtfs(x))
        batch_size, scale_rate, frequency, time = Sx.size()

        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)

        x = F.log_softmax(x, dim=1)
        return x
